In [1]:
import cv2
from tools_class import *
import threading
import numpy as np 
import matplotlib.pyplot as plt
import pyttsx3
import sys
import re
#import pyaudio
#import wave
#import matplotlib.pyplot as plt 
# import sys
# import time
# import re


global_res_text = 0
global_prev_text = 0

# Video-Thread
def video_thread(video, line_color, total_frames):
    
    global global_res_text

    plt.gray()

    #video, line_color, total_frames = user_input()
    print(total_frames)

    # declare some list and start values which are needed
    save_cnts = 0
    wait = 0
    # init class/values for while loop
    direct = FindDirection()
    
    #loop through all frames of video
    while video.isOpened():
        #Read a single frame
        ret, frame = video.read()
        if not ret:
            print("Error load frame")
            break
        
        #Create a Frame Object
        frame = FrameObject(frame)
        #get image from FrameObject
        img = frame.img

        #Create Segmemtation Object  
        seg = Segmentation(frame, line_color)
        #get the orientation line
        img_seg = seg.seg_orientation_lines()

        frame.build_center_line(img_seg)
        #used later to find the directions
        img = frame.make_block_over_center_line()
        
        # Get Values from FrameObject to Direction Object
        direct.get_values_from_frame_object(frame)
        direct.waiting(img)           #wait_for_compleat_cont
        direct.find_nearest_line(img_seg)
        direct.check_where_to_go(img_seg)
        res_text = direct.smooth_output()
        global_res_text = res_text
        frame.put_text_frame(res_text)    #text_in_frame
        frame_show = frame.draw_seg_orientationline(img_seg)     #draw_seg_orientationline, overlay_segmentatio
        

        cv2.imshow('Processed Frame',frame_show)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if video.get(cv2.CAP_PROP_POS_FRAMES) == total_frames:
            break
    # Release the video objects and close the windows
    video.release()
    cv2.destroyAllWindows()


# Audio-Thread
def audio_thread():

    global global_res_text
    global global_prev_text

    while video.isOpened():
        # Read a single frame
        ret, frame = video.read()
        if not ret:
            print("Error load frame")
            break
        
        # Überprüfen, ob sich das letzte Element geändert hat
        if global_res_text != global_prev_text:
            global_prev_text = global_res_text
            print("audio_txt", global_prev_text)
            audio_output(global_prev_text)
        else:
            print("prev_txt", global_prev_text)
        #time.sleep(0.1)


    # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if video.get(cv2.CAP_PROP_POS_FRAMES) == total_frames:
            break
    # Release the video objects and close the windows
        video.release()
        cv2.destroyAllWindows()


# Open the video file
video, line_color, total_frames = user_input()

# Threads starten
video_thread = threading.Thread(target=video_thread, args=(video, line_color, total_frames) )
audio_thread = threading.Thread(target=audio_thread)

video_thread.start()
audio_thread.start()

video_thread.join()
audio_thread.join()


You did a great Job!
2329
prev_txt 0
modulo 0


<Figure size 640x480 with 0 Axes>